In [2]:
import numpy as np
import pandas as pd

In [3]:
np.loadtxt('..\\data\\ml-100k\\ua.test', skiprows=0, delimiter='\t').astype('int32')

array([[        1,        20,         4, 887431883],
       [        1,        33,         4, 878542699],
       [        1,        61,         4, 878542420],
       ...,
       [      943,       570,         1, 888640125],
       [      943,       808,         4, 888639868],
       [      943,      1067,         2, 875501756]])

In [4]:
def load_data(path, delimiter='\t'):
    train = np.loadtxt(path+'ua.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'ua.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:, 0]).size #num of users
    n_i = np.unique(total[:, 1]).size #num of items

    train_data = np.zeros((n_u, n_i), dtype='float32')
    test_data = np.zeros((n_u, n_i), dtype='float32')

    for i in range(train.shape[0]):
        train_data[train[i][0]-1][train[i][1]-1] = train[i][2]
    
    for i in range(test.shape[0]):
        test_data[test[i][0]-1][test[i][1]-1] = test[i][2]

    return train_data, test_data

In [5]:
train_data, test_data = load_data(path='..\\data\\ml-100k\\')

In [6]:
a = 1601
sum(train_data[:, a])/np.count_nonzero(train_data[:, a])

3.3333333333333335

In [7]:
def average_item_rating(train_data):
    list_average_item_rating = [sum(train_data[:, i])/np.count_nonzero(train_data[:, i]) for i in range(train_data.shape[1])]
    return list_average_item_rating

In [8]:
list_average_item_rating = average_item_rating(train_data=train_data)

C:\Users\Theba\AppData\Local\Temp\ipykernel_16580\2928453520.py:2: RuntimeWarning: invalid value encountered in scalar divide
  list_average_item_rating = [sum(train_data[:, i])/np.count_nonzero(train_data[:, i]) for i in range(train_data.shape[1])]


In [9]:
list_average_item_rating[1183]

2.5555555555555554

In [10]:
def get_item_features(item_path): 
   i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

   movies = pd.read_csv(item_path, sep='|', names=i_cols,encoding='latin-1')
   genres = movies[['unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
   genres = genres.to_numpy()

   return genres
    

In [11]:
item_featuers = get_item_features('..\\data\\ml-100k\\u.item')

In [12]:
item_featuers[0]

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [13]:
import transformers

In [14]:
import torch

In [15]:
from torch.utils.data import DataLoader, Dataset

In [111]:
class CustomDataset(Dataset):
    def __init__(self, item_features: np.ndarray, labels: list) -> None:
        self.item_features = item_features
        self.labels = labels

    def __len__(self):
        return len(self.item_features)
    
    def __getitem__(self, idx) -> None:
        item = self.item_features[idx]
        label = self.labels[idx]

        return item, label

In [112]:
training_data = CustomDataset(item_features=item_featuers, labels=list_average_item_rating)

In [113]:
training_data_loader = DataLoader(training_data, batch_size=8, shuffle=True)

In [21]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1) # 1 label for regression

# Sample input description
description = "This product is amazing! It exceeded all my expectations."

# Tokenize input description
inputs = tokenizer(description, return_tensors="pt", max_length=512, truncation=True)

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted rating
predicted_rating = outputs.logits.item()

print("Predicted Rating:", predicted_rating)
    

c:\Users\Theba\miniconda3\envs\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Theba\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions

Predicted Rating: 0.11542760580778122


In [19]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [20]:
device

device(type='cpu')

In [ ]:
model.to(device)

In [22]:
from tqdm.auto import tqdm
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [123]:
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased', num_labels=1) # 1 label for regression


Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model c

In [124]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [145]:
num_epoch_community = 1

In [126]:
iter(training_data_loader)

In [127]:
model()

ValueError: You have to specify either input_ids or inputs_embeds

In [146]:
model.train()
for epoch in tqdm(range(num_epoch_community)):
    for batch in training_data_loader:
        outputs = model(input_ids = batch[0], labels=batch[1].float())
        print((outputs))
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

  0%|          | 0/1 [00:00<?, ?it/s]

SequenceClassifierOutput(loss=tensor(0.6187, grad_fn=<MseLossBackward>), logits=tensor([[3.0470],
        [3.0481],
        [2.9785],
        [3.0469],
        [2.8587],
        [3.2357],
        [3.0540],
        [2.9084]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(1.0123, grad_fn=<MseLossBackward>), logits=tensor([[2.5740],
        [2.2408],
        [2.1672],
        [2.1997],
        [2.5548],
        [2.5319],
        [2.4621],
        [2.3359]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.3907, grad_fn=<MseLossBackward>), logits=tensor([[2.6733],
        [2.9805],
        [3.2224],
        [2.8633],
        [2.7398],
        [2.9177],
        [3.0406],
        [2.8289]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=tensor(0.8665, grad_fn=<MseLossBackward>), logits=tensor([[3.2827],
        [3.2866],
        [3.2950],
  

KeyboardInterrupt: 

In [64]:
type(list_average_item_rating[0])

numpy.float64

In [98]:
next(iter(training_data_loader))

[tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
        dtype=torch.int32),
 tensor([2.2941, 2.0000, 3.5455, 3.7692, 3.4615, 4.1556, 2.3529, 3.1264],
        dtype=torch.float64)]

In [103]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [107]:
encoding = tokenizer("We are very happy to show you the Transformers library.", return_tensors='pt')
print(encoding)

{'input_ids': tensor([[  101,  2057,  2024,  2200,  3407,  2000,  2265,  2017,  1996, 19081,
          3075,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [108]:
model(**encoding)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1837]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [143]:
loss = torch.nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn((3, 5)).float()
output = loss(input, target)
print(output)

tensor(1.6690, grad_fn=<MseLossBackward>)
